In [0]:
# Original code licensed by:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# AlBERTo End to End (Fine-tuning + Predicting) with Cloud TPU

## Overview

**BERT**, or **B**idirectional **E**mbedding **R**epresentations from **T**ransformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks. The academic paper can be found here: https://arxiv.org/abs/1810.04805.

In particular we use this Notebook for fine-tuning **AlBERTo**, the first italian undertanding language model for Twitter Language.

**Note:**  You will need a GCP (Google Compute Engine) account and a GCS (Google Cloud 
Storage) bucket for this Colab to run.

Please follow the [Google Cloud TPU quickstart](https://cloud.google.com/tpu/docs/quickstart) for how to create GCP account and GCS bucket. 

You can learn more about Cloud TPU at https://cloud.google.com/tpu/docs.

## Instructions

<h3><a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>  &nbsp;&nbsp;Train on TPU</h3>

   1. Create a Cloud Storage bucket for your TensorBoard logs at http://console.cloud.google.com/storage and fill in the BUCKET parameter in the "Parameters" section below.
 
   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.

### Set up your TPU environment

In this section, you perform the following tasks:

*   Set up a Colab TPU running environment
*   Verify that you are connected to a TPU device
*   Upload your credentials to TPU to access your GCS bucket.

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

### Prepare and import BERT modules
​
With your environment configured, you can now prepare and import the BERT modules. The following step clones the source code from GitHub and import the modules from the source. 


In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from run_classifier import *
import modeling
import optimization
import tokenization

### Prepare for training

This next section of code performs the following tasks:

*  Specify task and download training data.
*  Specify ALBERTO pretrained model
*  Specify GS bucket, create output directory for model checkpoints and eval results.




In [0]:
TASK = 'SENTIPOLC_TASK3' #@param {type:"string"}

TASK_DATA_DIR = './'

BUCKET = 'alberto_files' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'


OUTPUT_DIR = 'gs://{}/{}/models/'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

#CONFIGURE AlBERTo MODEL
BERT_CONFIG_FILE = 'gs://alberto_files/alberto_model/bert_config.json' #@param {type:"string"}
VOCAB_FILE = 'gs://alberto_files/alberto_model/vocabulary_lower_case_128.txt' #@param {type:"string"}
INIT_CHECKPOINT = 'gs://alberto_files/alberto_model/model.ckpt-1000000'#@param {type:"string"}



Also we initilize our hyperprams, prepare the training data and initialize TPU config.

In [0]:
#SET THE PARAMETERS
TRAIN_BATCH_SIZE = 512
PREDICT_BATCH_SIZE = 512
EVAL_BATCH_SIZE = 512
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 10.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


In [0]:
#PREPARE TRAINING SENTENCES
!pip install ekphrasis
!pip install pandas
!pip install numpy

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import pandas as pd
import numpy as np

text_processor = TextPreProcessor (
    # terms that will be normalized
    normalize=[ 'url' , 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'] ,
    # terms that will be annotated
    annotate={"hashtag"} ,
    fix_html=True ,  # fix HTML tokens

    unpack_hashtags=True ,  # perform word segmentation on hashtags

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts = [ emoticons ]
)


In [0]:
#SETUP TRAINING AND TEST DATA
training_data = pd.read_csv(TASK_DATA_DIR+'alberto_sentipolc16_task1_trainig.tsv',delimiter="\t", header=None, names = ["id","class","star","message"], quoting=csv.QUOTE_NONE, error_bad_lines=False)
sentences = training_data.iloc[:,3]
labels = training_data.iloc[:,1]

#final examples training
examples = []

import re
i = 0
for s in sentences:
    s = s.lower()
    s = str(" ".join(text_processor.pre_process_doc(s)))
    s = re.sub(r"[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]", ' ', s)
    s = re.sub(r"\s+", ' ', s)
    s = re.sub(r'(\w)\1{2,}',r'\1\1', s)
    s = re.sub ( r'^\s' , '' , s )
    s = re.sub ( r'\s$' , '' , s )
    #print(s)
    examples.append([labels[i],s])
    i = i+1


examples = np.array(examples)

#final examples test
test_data = pd.read_csv(TASK_DATA_DIR+'alberto_sentipolc16_task3_test.tsv',delimiter="\t", names = ["id","message"], header=None, quoting=csv.QUOTE_NONE, error_bad_lines=False)
print(test_data.shape)
sentences_test = test_data.iloc[:,1]
test_ids = test_data.iloc[:,0]
sentences = test_data.iloc[:,1]

examples_test = []


i = 0
for s in sentences:
    s = s.lower()
    s = str(" ".join(text_processor.pre_process_doc(s)))
    s = re.sub(r"[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]", ' ', s)
    s = re.sub(r"\s+", ' ', s)
    s = re.sub(r'(\w)\1{2,}',r'\1\1', s)
    s = re.sub ( r'^\s' , '' , s )
    s = re.sub ( r'\s$' , '' , s )
    #print(s)
    examples_test.append([labels[i],s])
    i = i+1

examples_test = np.array(examples_test)

'''
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create InputExample's using the constructor provided in the BERT library.

    text_a is the text we want to classify, which in this case, is the Request field in our Dataframe.
    text_b is used if we're training a model to understand the relationship between sentences (i.e. is text_b a translation of text_a? Is text_b an answer to the question asked by text_a?). This doesn't apply to our task, so we can leave text_b blank.
    label is the label for our example, i.e. True, False

'''

f = lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[1], 
                                                                   text_b = None, 
                                                                   label = int(x[0]))

f2 = lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[1], 
                                                                   text_b = None, 
                                                                   label = 0)

train_examples = map(f,examples)
train_examples = list(train_examples)

test_examples = map(f2,examples_test)
test_examples = list(test_examples)

train_examples = np.array(train_examples)
test_examples = np.array(test_examples)

print(test_examples.shape)

In [0]:
#Labels used for annotating sentences
label_list = [0, 1]

In [0]:
#Test data just created
for r in test_examples:
  print(r.text_a)

In [0]:
#inizialize parameters
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)+1
num_warmup_steps = int(NUM_TRAIN_EPOCHS * WARMUP_PROPORTION)
print(num_train_steps)
print(num_warmup_steps)

#Inizialize the tokenizer
tokenizer = tokenization.FullTokenizer(VOCAB_FILE, do_lower_case=True)
tokenizer.tokenize("Che bella giornata oggi! Stiamo procedendo bene o no? <url>")

# Fine-tune and Run Predictions on a pretrained BERT Model

This section demonstrates fine-tuning from a pre-trained BERT TF Hub module and running predictions.


In [0]:
BERT_CONFIG= modeling.BertConfig.from_json_file(BERT_CONFIG_FILE)


model_fn = model_fn_builder(
  bert_config=BERT_CONFIG,
  num_labels=len(label_list),
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True
)

estimator = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


In [0]:
train_features = convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

print("***** Running training *****")
print("  Num examples = %d", len(train_examples))
print("  Num labels = %d", len(label_list))
print("  Batch size = %d", TRAIN_BATCH_SIZE)
print("  Num steps = %d", num_train_steps)

train_input_fn = input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

print('***** Started training at {} *****'.format(datetime.datetime.now()))
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('***** Finished training at {} *****'.format(datetime.datetime.now()))


In [0]:
# MODEL PREDICTIONS
input_features = convert_examples_to_features(
      test_examples,label_list, MAX_SEQ_LENGTH, tokenizer)
  
print(len(list(input_features)))  
print(len(test_ids))
print(len(sentences_test))
print('***** Started predictions at {} *****'.format(datetime.datetime.now()))

# Eval will be slightly WRONG on the TPU because it will truncate
# the last batch.

predict_input_fn = input_fn_builder(
    features=input_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)


predictions = estimator.predict(predict_input_fn)

print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))


  
#SAVE IN BUCKLET RESULTS AND PRINT THEM
output_eval_file = os.path.join(OUTPUT_DIR, "alberto_sentipolc16_task3_results.tsv")
with tf.gfile.GFile(output_eval_file, "w") as writer:
  print("***** Results *****")
  for example, prediction, id in zip(sentences_test, predictions, test_ids):
    print('\t prediction:%s \t id:%s \t text_a: %s' % ( np.argmax(prediction['probabilities']),str(id),str(example) ) )
    writer.write("%s \t %s\n" % (str(id), np.argmax(prediction['probabilities'])) )
